# Source localization pipeline - Stage1
- freesurfer
- .fif-File preparations
 
 


#### Author
Rudi Kreidenhuber <Rudi.Kreidenhuber@gmail.com>
#### License
BSD (3-clause)

## Subject selection

In [4]:

from src import SubjectDropDowner
import os
import glob
import mne

input_dir = "/home/meg/Schreibtisch/new_patients"

def get_subject_list(input_dir = input_dir) -> list:
    analist = glob.glob(os.path.join(input_dir, "*"))
    analist = [os.path.basename(f) for f in analist if os.path.isdir(f)]
    fiflist = [os.path.basename(f) for f in glob.glob(os.path.join(input_dir, "*trans_tsss.fif"))]
    fiflist = [f.split("_")[0] for f in fiflist]
    subjectlist = set(fiflist + analist)
    subjectlist = sorted([f for f in subjectlist])
    return subjectlist

subjectlist = get_subject_list()

dl = SubjectDropDowner.SubjectDropDowner(subjectlist)
drop_menu = dl.create_subject_dropdown_widget()
print(f"\n\nThe following patients/ subjects are available:")
drop_menu



The following patients/ subjects are available:


# .fif - File preprocessing
- Files are being filtered, downsampled and concatenated
- Files without head transposition and artifact correction via tsss are omitted

In [7]:
subject = dl.get_subject_dropdown().value

if not subject.startswith("sub-"):
    ject = subject
    subject = "sub-" + subject
else:
    subject = subject
    ject = subject.replace("sub-", "")

print(f"Ject: {ject}, Subject: {subject}")

if not os.path.isdir(os.path.join(input_dir, "processed")):
    os.mkdir(os.path.join(input_dir, "processed"))

output_dir = os.path.join(input_dir, "processed")
target_file = os.path.join(output_dir, str(subject + "_prepped.fif"))

# Make sure we haven't already processed this subject
if os.path.isfile(target_file):
    print(f"\n\nThe file {target_file} already exists, aborting.\n\n")
    raise(FileExistsError)

# Configuration 
# Filter and resample
l_freq: float = 0.1         # lower pass-band edge
h_freq: float = 50.         # higher pass-band edge
fir_design: str = "firwin"  # Filter design method
s_freq: int = 300           # target sampling frequency
# ECG artifact correction
ecg_channel = "ECG003"
n_grad: int =1
n_mag: int = 1
n_eeg: int = 1


# Get all .fif files of the subject
raws = glob.glob(os.path.join(input_dir, "*_trans_tsss.fif"))
raws = [f for f in raws if ject in f]
print(f"The following raw files were found for preprocessing:\n{raws}")

prep_raws = []
for r in raws:
    raw = mne.io.read_raw(r)
    # filter
    raw.load_data()
    raw = raw.filter(l_freq, h_freq, fir_design=fir_design)
    # downsample
    if not raw.info["sfreq"] == s_freq:
        raw = raw.resample(s_freq)
    prep_raws.append(raw)
    del raw

# concatenate
try:
    raw = mne.concatenate_raws(prep_raws)
    raw.save(target_file, overwrite=False)
except Exception as e:
    print(f"\n\nSomething went wrong, when trying to concatenate the raw files:\n\nError: {e}")

del prep_raws

Ject: VA09051993, Subject: sub-VA09051993
The following raw files were found for preprocessing:
['/home/meg/Schreibtisch/new_patients/VA09051993_block2_trans_tsss.fif']
Opening raw data file /home/meg/Schreibtisch/new_patients/VA09051993_block2_trans_tsss.fif...
    Range : 21000 ... 1243999 =     21.000 ...  1243.999 secs
Ready.
Reading 0 ... 1222999  =      0.000 ...  1222.999 secs...


/tmp/ipykernel_208710/1106302612.py:43: RuntimeWarning: This filename (/home/meg/Schreibtisch/new_patients/VA09051993_block2_trans_tsss.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw(r)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 33001 samples (33.001 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 370 out of 370 | elapsed:   20.3s finished


Writing /home/meg/Schreibtisch/new_patients/processed/sub-VA09051993_prepped.fif


/tmp/ipykernel_208710/1106302612.py:56: RuntimeWarning: This filename (/home/meg/Schreibtisch/new_patients/processed/sub-VA09051993_prepped.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(target_file, overwrite=False)


Closing /home/meg/Schreibtisch/new_patients/processed/sub-VA09051993_prepped.fif
[done]


# Freesurfer and hippocampal segmentation

In [ ]:
def get_dicom_file(subject) -> str:
    if not os.path.isdir(os.path.join(input_dir, subject)):
        print(f"No anatomical data found for {subject}, aborting\n\n")
        raise(FileNotFoundError)
    else:
        dicom_path = os.path.join(input_dir, subject, "1*", "100*", "1*", "*")
        dicom = str(glob.glob(dicom_path, recursive=True)[0])
        dicom = os.path.abspath(dicom)
        return dicom

def path_to_wsl(file) -> str:
    file = file.replace("\\", "/")
    return file.replace("c:", "/mnt/c")

def get_dicom_path(subject) -> str:
    dicom = get_dicom_file(subject)
    dicom = path_to_wsl(dicom)
    return dicom

def get_watershed_comand(subject) -> str:
    return f'python -c "import mne; mne.bem.make_watershed_bem(subject=\'{subject}\')"'

def get_recon_all_command(ject) -> str:
    dicom = get_dicom_path(ject)
    subject = "sub-" + ject
    command = f"recon-all -s {subject} -i {dicom} -all && segmentHA_T1.sh {subject} && {get_watershed_comand(subject)}" 
    return command

command = get_recon_all_command(ject)

print(f"\n\nExecute the following command in the WSL shell:\n \
      (This will take hours...)\n\n{command}\n\n")

print(f"Meanwhile use brainstorm3 to mark and group spikes on the \
following file:\n\n {os.path.abspath(target_file)}\n\n")

event_file_name = target_file.split(".fif")[0] + "_Events.fif"
print(f"Save the eventfile in the same directory as the raw file as: {os.path.basename(event_file_name)}")
